In [ ]:
key=""

In [2]:
from pydantic_ai import Agent
import os

os.environ["GEMINI_API_KEY"] = key

agent = Agent(
    "gemini-1.5-pro",
    system_prompt="Be concise, reply with one sentence.",
)


result = await agent.run('Where does "hello world" come from?')
print( result.output)

"Hello, World!" originated from a 1972 Bell Laboratories internal memorandum by Brian Kernighan, *A Tutorial Introduction to the Language B*,  where it was used to illustrate a simple program.



# 流式响应

agent.run_stream


In [ ]:
from pydantic_ai import Agent
import os

os.environ["GEMINI_API_KEY"] = key

agent = Agent(
    "gemini-1.5-pro",
    system_prompt="Be concise, reply with one sentence.",
)


async with agent.run_stream('Where does "hello world" come from?') as response:
    print(await response.get_output())

"Hello, World!" originated from a 1972 Bell Laboratories internal memorandum by Brian Kernighan, *A Tutorial Introduction to the Language B*.



# 反思与自我纠正

验证请求，并且要求大模型重新思考


In [14]:
from typing import Union
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext, ModelRetry


class Success(BaseModel):
    sql_query: str


class InvalidRequest(BaseModel):
    error_message: str

Output = Union[Success, InvalidRequest]
agent = Agent[Output](
    "gemini-1.5-pro",
    output_type=Output,  # type: ignore
    system_prompt='Generate PostgreSQL flavored SQL queries based on user input.',
)

@agent.output_validator
async def validate_sql(ctx: RunContext, output: Output) -> Output:
    if isinstance(output, InvalidRequest):
        return output
    try:
        print(f'Executing query: {output.sql_query}')
        if output.sql_query.startswith('SELECT'):
            # 要求小写SQL
            raise ModelRetry('use lower case')
    except Exception as e:
        raise ModelRetry(f'Invalid query: {e}') from e
    else:
        return output


result = await agent.run(
    'get me users who were last active yesterday.'
)
print(result.output)


Executing query: SELECT * FROM users WHERE last_active = CURRENT_DATE - INTERVAL '1 day';
Executing query: select * from users where last_active = current_date - interval '1 day';
sql_query="select * from users where last_active = current_date - interval '1 day';"


# token

查看、限制 Token 的使用


In [ ]:
from pydantic_ai import Agent
from pydantic_ai.usage import UsageLimits


agent = Agent(
    "gemini-1.5-pro",
    system_prompt="Be concise, reply with one sentence.",
)


result_sync = await agent.run(
    'Where does "hello world" come from?',
    usage_limits=UsageLimits(response_tokens_limit=44),
)
result_sync.output

'"Hello, World!" originated from Brian Kernighan\'s 1972 tutorial *A Tutorial Introduction to the Language B*.\n'

## 统计使用情况


In [10]:
result_sync.usage()

Usage(requests=1, request_tokens=17, response_tokens=29, total_tokens=46, details=None)

## 超出限制异常


In [11]:
try:
    result_sync = await agent.run(
    'Where does "hello world" come from?',
    usage_limits=UsageLimits(response_tokens_limit=10),
)
except UsageLimitExceeded as e:
    print(e)

Exceeded the response_tokens_limit of 10 (response_tokens=44)


# 编程式 system_prompt

通过@agent.system_prompt 给 agent 添加 system_prompt


In [7]:
from pydantic_ai import Agent, RunContext
import os
from datetime import date

os.environ["GEMINI_API_KEY"] = key

agent = Agent(
    "gemini-1.5-pro",
    system_prompt="Be concise, reply with one sentence.",
)


@agent.system_prompt  
def add_the_users_name(ctx: RunContext[str]) -> str:
    return f"The user's name is {ctx.deps}."


@agent.system_prompt
def add_the_date() -> str:  
    return f'The date is {date.today()}.'


result = await agent.run('What is the date?', deps='Frank')
result.output

'Frank, today is April 25, 2025.\n'

# 连续记忆

记住上一次对话

message_history


In [3]:
from pydantic_ai import Agent
import os

os.environ["GEMINI_API_KEY"] = key

agent = Agent(
    "gemini-1.5-pro",
    system_prompt="Be concise, reply with one sentence.",
)

result1 = await agent.run('Who was Albert Einstein?')
result1.output

'Albert Einstein was a theoretical physicist widely regarded as one of the most influential scientists of all time, best known for developing the theory of relativity.\n'

In [4]:

# Second run, passing previous messages
result2 = await agent.run(
    'What was his most famous equation?',
    message_history=result1.new_messages(),  
)
result2.output

'His most famous equation is E=mc², which describes the relationship between energy (E) and mass (m), where c represents the speed of light.\n'